<a href="https://colab.research.google.com/github/KamilBienias/data-science/blob/main/kursPawe%C5%82Krakowiak/neural-network-course/03_keras/01_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Model sekwencjny (Sequential Model):
1. [Wprowadzenie - pierwszy model](#a0)
2. [Funkcje aktywacji](#a1)
3. [Kompilacja modelu](#a2)
4. [Trenowanie modelu](#a3)
5. [Przykład - klasyfikacja binarna](#a4)
6. [Przykład - klasyfikacja wieloklasowa](#a5)
7. [Przykład - regresja](#a6)



Załadowanie biblioteki tensorflow


In [2]:
# ######################################################
# Etap 7. Odcinek: Biblioteka Keras - Wprowadzenie.

%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import pandas as pd
import plotly.express as px
tf.__version__

'2.4.1'

### <a name='a0'></a>Wprowadzenie - pierwszy model
Model sekwencyjny to nic innego jak liniowy stos warstw.

In [3]:
# utworzenie instancji klasy Sequential
from tensorflow.keras.models import Sequential

model = Sequential()
print(model)

Podstawowym elementem składowym modelu są warstwy. Aby dodać najbardziej standardową warstwę - warstwę gęsto połączoną należy użyć warstwy **Dense**. Aby dodać warstwę do modelu należy użyć metody *.add()*

In [4]:
from tensorflow.keras.layers import Dense

# Nie trzeba jawnie definiować warstwy wejściowej.
# Wystarczy przy pierwszej podanej wartstwie podać parametr input_shape,
# nasze dane wejściowe niech mają 10 cech (kolumn).
# Parametr units to liczba neuronów w wartstwie ukrytej. 
model.add(Dense(units=4, input_shape=(10,)))

Wyświetlenie podsumowania modelu: metoda *.summary()*

In [5]:
# None to ilość próbek, czyli ilość danych wejściowych. 
# Ilość parametrów w dense_1 to 44, bo wzór:
# Param = (input_shape + 1) * units = (10+1)*4 = 44
# Tą 1 dodaje się bo bias
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 44        
Total params: 44
Trainable params: 44
Non-trainable params: 0
_________________________________________________________________


Dodanie kolejnej warstwy

In [6]:
# do kolejnej wartsty nie podawać rozmiaru danych wejściowych,
# bo keras się domyśli. Ta druga warstwa ukryta ma 2 neurony, bo units=2
model.add(Dense(units=2))

model.summary()
# teraz w dense_2 jest (4+1)*2=10 parametrów

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 44        
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 10        
Total params: 54
Trainable params: 54
Non-trainable params: 0
_________________________________________________________________


### <a name='a1'></a>Funkcje aktywacji
Istotnym elemenem sieci neuronowych jest dobór odpowiednich funkcji aktywacji. Funkcje aktywacji jak sama nazwa wskazuje są odpowiedzialne za aktywowanie odpowiednich neuronów podczas procesu uczenia.

Jeżeli nie określimy podczas dodawania warstwy funkcji aktywacji, domyślnie stosowana jest liniowa funkcja aktywacji, tzn. $a(x)=x$ czyli tak naprawdę nic się nie dzieje.

Warstwa z liniową funkcją aktywacji może uczyć się tylko liniowych przekształceń danych wejściowych. Dlatego stosuje się różne funkcje aktywacji aby rozwiązywać problemy nieliniowe.

Dokumentacja keras z funkcjami aktwacji:
[Keras: Funkcje aktywacji](https://keras.io/activations/)

In [7]:
# importuje funkcję liniową a(x) (identycznościową)
from tensorflow.keras.activations import linear

random_data = sorted(np.random.randn(200))
data = pd.DataFrame({'data': random_data, 'linear': linear(random_data)})
data.head()

,data,linear
0,-2.762310,-2.762310
1,-2.707523,-2.707523
2,-2.125510,-2.125510
3,-1.932152,-1.932152
4,-1.905046,-1.905046


In [8]:
px.line(data, x='data', y='linear', width=800, range_y=[-2, 2])

In [9]:
from tensorflow.keras.activations import sigmoid

data = pd.DataFrame({'data': random_data, 'sigmoid': sigmoid(random_data)})
data.head()

,data,sigmoid
0,-2.762310,0.059395
1,-2.707523,0.062531
2,-2.125510,0.106642
3,-1.932152,0.126513
4,-1.905046,0.129538


In [10]:
px.line(data, x='data', y='sigmoid', width=800, range_y=[-0.5, 1.5])

In [11]:
from tensorflow.keras.activations import relu

data = pd.DataFrame({'data': random_data, 'relu': relu(random_data)})
data.head()

,data,relu
0,-2.762310,0.0
1,-2.707523,0.0
2,-2.125510,0.0
3,-1.932152,0.0
4,-1.905046,0.0


In [12]:
px.line(data, x='data', y='relu', width=800, range_y=[-0.5, 1.5])

In [13]:
from tensorflow.keras.activations import tanh

data = pd.DataFrame({'data': random_data, 'tanh': tanh(random_data)})
data.head()

,data,tanh
0,-2.762310,-0.992057
1,-2.707523,-0.991141
2,-2.125510,-0.971901
3,-1.932152,-0.958907
4,-1.905046,-0.956667


In [14]:
px.line(data, x='data', y='tanh', width=800, range_y=[-1.5, 1.5])

In [15]:
# model od nowa
model = Sequential()
# input_shape musi być jako tupla bo ma być iterowalna.
# U nas 10 cech wejściowych
model.add(Dense(units=8, activation='relu', input_shape=(10,)))
# wartstwa output ma 1 neuron
model.add(Dense(units=1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 8)                 88        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 9         
Total params: 97
Trainable params: 97
Non-trainable params: 0
_________________________________________________________________


### <a name='a2'></a>Kompilacja modelu
Przed rozpoczęciem trenowania sieci należy odpowiednio skonfigurować proces uczenia. W tym kroku określamy:
* rodzaj optymalizatora ([Keras - Optymalizatory](https://keras.io/optimizers/)) 
* funkcję straty ([Keras - Funkcje Straty](https://keras.io/losses/))
* metryki, które będziemy obserwować podczas trenowania sieci ([Keras - Metryki](https://keras.io/metrics/))

In [16]:
# klasyfikacja binarna. Wtedy koniecznie loss='binary_crossentropy'
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# klasyfikacja wieloklasowa. Wtedy koniecznie loss='categorical_crossentropy'
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# regresja. Wtedy można loss='mse' lub loss='mae'. 
model.compile(optimizer='rmsprop',
              loss='mse')

### <a name='a3'></a>Trenowanie modelu
Za dane wejściowe do modelu należy przekazać Numpy arrays:
* **epochs** - krotność przejścia danych przez sieć w procesie uczenia
* **batch_size** - rozmiar wsadu po którym następuje aktualizacja wag
* **validation_split** - część danych treningowych, które zostaną wykorzystane jako zbiór walidacyjny
* **validation_data** - (x_val, y_val) - dane wykorzystane do walidacji modelu

In [ ]:
# model.fit(data, labels, epochs=10, batch_size=32)
# model.fit(data, labels, epochs=10, batch_size=32, validation_split=0.2)
# model.fit(data, labels, epochs=10, batch_size=32, validation_data=(x_val, y_val))

### <a name='a4'></a> Przykład - klasyfikacja binarna

In [17]:
# ######################################################
# Etap 7. Odcinek: Praca z modelami w bibliotece Keras.

# 1000 list po 150 liczb z rozkładu N(0,1) w każdej liście
data = np.random.randn(1000, 150)
# są dwie klasy 0 i 1, bo klasyfikacja binarna
labels = np.random.randint(2, size=(1000, 1))

print(data.shape)
print(labels.shape)

(1000, 150)
(1000, 1)


In [18]:
data[:3]

array([[ 0.22524385,  0.82340765, -0.31145081,  1.62016711,  0.01979965,
        -0.07805478, -0.25551812, -0.16139485,  1.53153695,  0.72862926,
         1.97308309,  0.30906109,  0.36024795,  0.58845595,  0.21010228,
        -0.55102812, -0.62760446, -1.02834831,  0.9143662 ,  0.55002389,
        -0.1548795 , -1.22930835, -0.38467203,  1.02857361,  1.4787169 ,
        -0.11347062,  0.24086258, -1.30372195,  0.36832882,  0.06242807,
         0.33889059,  0.04232403,  0.94504531,  0.31377791,  0.23326129,
         1.3153132 ,  0.70469214,  0.72937925, -0.73650206,  1.84928624,
         0.78864103,  0.27848125, -0.67908961, -0.64089373,  0.27226556,
        -1.24342683,  0.31192525,  0.1162281 ,  0.57138901, -1.11677529,
        -0.83182872, -1.30713301,  0.02902337,  0.21589382,  0.75897637,
         0.76083362,  0.26042158,  0.18907206, -1.7853969 ,  2.17388078,
        -1.76483782,  0.37121577,  1.72725582,  0.41280171, -1.62579231,
        -2.032144  , -0.85958432,  2.47385891, -1.1

In [20]:
# klasy 0 lub 1
labels[:10]

array([[1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0]])

In [21]:
model = Sequential()
# często liczba neuronów w warstwie jest potęgą 2.
# activation='relu' aby model uczył się nieliniowych rozwiązań.
# Mamy 150 zmiennych (kolumn) czyli wartstwa wejściowa ma 150 wejść
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
# warstwa wyjściowa ma 1 neuron bo chcemy wartość p-twa przynależności do danej klasy.
# activation='sigmoid' zamieni dane, które otrzyma na wartości p-twa
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',  # koniecznia ta funkcja straty
              metrics=['accuracy'])

# trenowanie modelu. Metoda fit zwraca obiekt klasy History.
# Na dole jest info o tym.
model.fit(data, labels, epochs=20)

Epoch 1/20
32/32 [==============================] - 1s 1ms/step - loss: 0.8341 - accuracy: 0.5043
Epoch 2/20
32/32 [==============================] - 0s 1ms/step - loss: 0.7230 - accuracy: 0.5470
Epoch 3/20
32/32 [==============================] - 0s 1ms/step - loss: 0.6564 - accuracy: 0.6074
Epoch 4/20
32/32 [==============================] - 0s 1ms/step - loss: 0.6103 - accuracy: 0.6306
Epoch 5/20
32/32 [==============================] - 0s 1ms/step - loss: 0.5587 - accuracy: 0.7104
Epoch 6/20
32/32 [==============================] - 0s 1ms/step - loss: 0.5327 - accuracy: 0.7548
Epoch 7/20
32/32 [==============================] - 0s 1ms/step - loss: 0.5226 - accuracy: 0.7703
Epoch 8/20
32/32 [==============================] - 0s 1ms/step - loss: 0.4892 - accuracy: 0.8059
Epoch 9/20
32/32 [==============================] - 0s 1ms/step - loss: 0.4626 - accuracy: 0.8307
Epoch 10/20
32/32 [==============================] - 0s 1ms/step - loss: 0.4384 - accuracy: 0.8630
Epoch 11/20
32/32 [

In [ ]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# mogę dodać rozmiar wsadu czyli batch_size.
# Wagi będą aktualizowane co 30 próbek.
model.fit(data, labels, epochs=20, batch_size=30)

Train on 1000 samples
Epoch 1/20
1000/1000 [==============================] - 1s 504us/sample - loss: 0.8355 - accuracy: 0.5140
Epoch 2/20
1000/1000 [==============================] - 0s 71us/sample - loss: 0.7114 - accuracy: 0.5630
Epoch 3/20
1000/1000 [==============================] - 0s 70us/sample - loss: 0.6549 - accuracy: 0.6020
Epoch 4/20
1000/1000 [==============================] - 0s 74us/sample - loss: 0.6130 - accuracy: 0.6460
Epoch 5/20
1000/1000 [==============================] - 0s 73us/sample - loss: 0.5789 - accuracy: 0.6900
Epoch 6/20
1000/1000 [==============================] - 0s 70us/sample - loss: 0.5491 - accuracy: 0.7310
Epoch 7/20
1000/1000 [==============================] - 0s 70us/sample - loss: 0.5218 - accuracy: 0.7550
Epoch 8/20
1000/1000 [==============================] - 0s 75us/sample - loss: 0.4976 - accuracy: 0.7780
Epoch 9/20
1000/1000 [==============================] - 0s 65us/sample - loss: 0.4733 - accuracy: 0.8080
Epoch 10/20
1000/1000 [=========

In [22]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 20% danych treningowych będzie zbiorem walidacyjnym.
# Doszły wartości val_loss i val_accuracy. Te metryki bardziej nas interesują,
# bo to wyniki na danych których model nie widział
model.fit(data, labels, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20
25/25 [==============================] - 1s 19ms/step - loss: 0.8141 - accuracy: 0.5118 - val_loss: 0.7672 - val_accuracy: 0.5100
Epoch 2/20
25/25 [==============================] - 0s 3ms/step - loss: 0.7107 - accuracy: 0.5681 - val_loss: 0.7556 - val_accuracy: 0.5000
Epoch 3/20
25/25 [==============================] - 0s 3ms/step - loss: 0.6503 - accuracy: 0.6312 - val_loss: 0.7510 - val_accuracy: 0.4850
Epoch 4/20
25/25 [==============================] - 0s 3ms/step - loss: 0.6298 - accuracy: 0.6563 - val_loss: 0.7502 - val_accuracy: 0.5000
Epoch 5/20
25/25 [==============================] - 0s 3ms/step - loss: 0.5889 - accuracy: 0.6915 - val_loss: 0.7506 - val_accuracy: 0.4900
Epoch 6/20
25/25 [==============================] - 0s 3ms/step - loss: 0.5670 - accuracy: 0.7266 - val_loss: 0.7526 - val_accuracy: 0.4950
Epoch 7/20
25/25 [==============================] - 0s 3ms/step - loss: 0.5486 - accuracy: 0.7446 - val_loss: 0.7561 - val_accuracy: 0.4950
Epoch 8/20
25/25 [=

In [23]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# verbose=0 pozwala unikać tego co się wyświetla i zwraca tylko obiekt History
model.fit(data, labels, epochs=20, batch_size=32, validation_split=0.2, verbose=0)

In [25]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# przypisuje wynik metod fit do zmiennej history
history = model.fit(data, labels, epochs=20, batch_size=32, validation_split=0.2, verbose=0)

In [26]:
metrics = history.history
metrics.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [27]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(data, labels, epochs=20, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/20
25/25 [==============================] - 1s 9ms/step - loss: 0.7926 - accuracy: 0.5036 - val_loss: 0.7478 - val_accuracy: 0.5600
Epoch 2/20
25/25 [==============================] - 0s 3ms/step - loss: 0.7164 - accuracy: 0.5498 - val_loss: 0.7443 - val_accuracy: 0.5350
Epoch 3/20
25/25 [==============================] - 0s 3ms/step - loss: 0.6667 - accuracy: 0.6023 - val_loss: 0.7433 - val_accuracy: 0.5200
Epoch 4/20
25/25 [==============================] - 0s 3ms/step - loss: 0.6218 - accuracy: 0.6683 - val_loss: 0.7428 - val_accuracy: 0.5250
Epoch 5/20
25/25 [==============================] - 0s 3ms/step - loss: 0.5765 - accuracy: 0.7123 - val_loss: 0.7441 - val_accuracy: 0.5250
Epoch 6/20
25/25 [==============================] - 0s 3ms/step - loss: 0.5636 - accuracy: 0.7304 - val_loss: 0.7478 - val_accuracy: 0.5200
Epoch 7/20
25/25 [==============================] - 0s 4ms/step - loss: 0.5283 - accuracy: 0.7828 - val_loss: 0.7525 - val_accuracy: 0.5000
Epoch 8/20
25/25 [==

In [29]:
# tworzy dane testowe
# 5 próbek po 150 cech w każdej
test_data = np.random.randn(5, 150)
# 5 labeli
test_labels = np.random.randint(2, size=(5, 1))

In [30]:
# funkcja predict zwraca p-two przynależności do danej klasy.
# Tam gdzie wartości > 0.5 to przypisujemny do klasy 1
# a tam gdzie < 0.5 to do klasy 0. Czyli na przykład [0.17643541] to klasa 0,
# a [0.17643541] to klasa 1.
model.predict(test_data)*

array([[0.17643541],
       [0.32152855],
       [0.22700277],
       [0.47679415],
       [0.64433014]], dtype=float32)

In [31]:
# funkcja predict działa tak samo jak predict_proba (predict probabilities)
model.predict_proba(test_data)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning:

`model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.



array([[0.17643541],
       [0.32152855],
       [0.22700277],
       [0.47679415],
       [0.64433014]], dtype=float32)

In [32]:
model.predict_classes(test_data)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning:

`model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).



array([[0],
       [0],
       [0],
       [0],
       [1]], dtype=int32)

### <a name='a5'></a> Przykład - klasyfikacja wieloklasowa

In [33]:
data = np.random.random((1000, 150))
# teraz 10 klas
labels = np.random.randint(10, size=(1000, 1))

In [34]:
print(data.shape)
print(labels.shape)

(1000, 150)
(1000, 1)


In [35]:
data[:3]

array([[0.78323415, 0.0255039 , 0.01179862, 0.66858866, 0.30826742,
        0.67047761, 0.36563791, 0.65941634, 0.20300033, 0.43433942,
        0.46892031, 0.68620496, 0.34510762, 0.97361419, 0.20054708,
        0.80637089, 0.17701616, 0.05817144, 0.82183755, 0.89656723,
        0.98583075, 0.16589413, 0.18438051, 0.95165059, 0.20370399,
        0.41055979, 0.22586619, 0.56380348, 0.65537096, 0.83458969,
        0.94093739, 0.38888292, 0.98088405, 0.71720742, 0.67345047,
        0.01512395, 0.73372856, 0.36255462, 0.98120677, 0.54213643,
        0.02797919, 0.61688359, 0.68847936, 0.09077178, 0.02470348,
        0.55838436, 0.45848496, 0.3293772 , 0.72819366, 0.68964172,
        0.71728084, 0.21927016, 0.83995851, 0.59060747, 0.96854564,
        0.00624297, 0.78349805, 0.67438311, 0.71395015, 0.54315193,
        0.36796653, 0.4714374 , 0.37379809, 0.64677079, 0.93646138,
        0.81748603, 0.47882899, 0.32775483, 0.38225223, 0.16614604,
        0.64430283, 0.55020776, 0.27467536, 0.09

In [36]:
labels[:10]

array([[0],
       [6],
       [7],
       [3],
       [2],
       [5],
       [7],
       [2],
       [6],
       [1]])

In [37]:
from tensorflow.keras.utils import to_categorical
# funkcja to_categorical przekonwertuje labele na listy zero-jedynkowe
labels = to_categorical(labels, num_classes=10)
labels

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [38]:
# na przykład gdy druga lista wynosi [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]
# to oznacza liczbę 6
labels[1]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32)

In [39]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
# neuronów tyle ile mamy klas.
# Funkcja softmax pozwala zwrócić p-two przynależności do tych klas
model.add(Dense(units=10, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, batch_size=32, epochs=30, validation_split=0.2)

Epoch 1/30
25/25 [==============================] - 1s 9ms/step - loss: 2.3335 - accuracy: 0.1110 - val_loss: 2.3003 - val_accuracy: 0.1300
Epoch 2/30
25/25 [==============================] - 0s 3ms/step - loss: 2.2939 - accuracy: 0.1270 - val_loss: 2.3057 - val_accuracy: 0.1150
Epoch 3/30
25/25 [==============================] - 0s 3ms/step - loss: 2.2742 - accuracy: 0.1199 - val_loss: 2.3083 - val_accuracy: 0.1100
Epoch 4/30
25/25 [==============================] - 0s 4ms/step - loss: 2.2738 - accuracy: 0.1362 - val_loss: 2.3138 - val_accuracy: 0.0950
Epoch 5/30
25/25 [==============================] - 0s 3ms/step - loss: 2.2763 - accuracy: 0.1368 - val_loss: 2.3157 - val_accuracy: 0.1250
Epoch 6/30
25/25 [==============================] - 0s 3ms/step - loss: 2.2499 - accuracy: 0.1542 - val_loss: 2.3073 - val_accuracy: 0.1150
Epoch 7/30
25/25 [==============================] - 0s 3ms/step - loss: 2.2509 - accuracy: 0.1519 - val_loss: 2.3108 - val_accuracy: 0.1200
Epoch 8/30
25/25 [==

In [40]:
test_data = np.random.random((10, 150))
# dla każdej próbki wektor długości 10 z p-twami przynależności do poszczególnej klasy.
# Tam gdzie p-two najwyższe, to do tej klasy zostnie przypisana próbka.
model.predict(test_data)

array([[0.07045627, 0.23586144, 0.02904801, 0.08006576, 0.26272058,
        0.04720489, 0.1459829 , 0.04409807, 0.04077771, 0.0437843 ],
       [0.04768011, 0.22445527, 0.08131329, 0.14248304, 0.20981674,
        0.0523602 , 0.10710092, 0.05029355, 0.03251868, 0.05197823],
       [0.01628052, 0.2168705 , 0.170518  , 0.0571672 , 0.06755725,
        0.09694082, 0.20739794, 0.01600729, 0.0711379 , 0.0801226 ],
       [0.11671582, 0.06626239, 0.15857537, 0.08358145, 0.20708127,
        0.06999322, 0.15911245, 0.03758769, 0.03963159, 0.06145867],
       [0.04587272, 0.12613493, 0.08729503, 0.10691224, 0.13946286,
        0.19022901, 0.12736343, 0.04608225, 0.05661784, 0.07402962],
       [0.1581612 , 0.06737335, 0.07602661, 0.11582807, 0.1084618 ,
        0.07639979, 0.07004075, 0.13598773, 0.06911908, 0.12260166],
       [0.06137703, 0.16819672, 0.14443237, 0.08216718, 0.04740322,
        0.21006203, 0.04515247, 0.09478693, 0.02699184, 0.1194301 ],
       [0.0789526 , 0.15648599, 0.1524376

In [41]:
# przewidziane klasy
model.predict_classes(test_data)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning:

`model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).



array([4, 1, 1, 4, 5, 0, 5, 1, 0, 3])

### <a name='a6'></a> Przykład - regresja

In [42]:
# dane to 1000 próbek i 150 liczb w każdej próbce z rozkładu jednostajnego U(0,1) 
# czyli z przedziału (0,1)
data = np.random.random((1000, 150))
# labele to 1000 liczb z rozkładu jednostajnego U(0,50), czyli z przedziału (0,50)
# Czyli już to są ciągłe wartości
labels = 50 * np.random.random(1000)

In [44]:
# pierwsze 3 próbki, po 150 cech w każdej próbce
data[:3]

array([[0.93039107, 0.87871847, 0.87943923, 0.23029392, 0.92462355,
        0.7471006 , 0.49802824, 0.96498018, 0.70362733, 0.8592457 ,
        0.2722082 , 0.05040313, 0.7624297 , 0.38987293, 0.79006158,
        0.26088318, 0.8229753 , 0.73231732, 0.30395526, 0.00131049,
        0.62140726, 0.56842562, 0.97605632, 0.59144721, 0.5107196 ,
        0.52944263, 0.23738964, 0.01461417, 0.23769062, 0.49292143,
        0.26117576, 0.6788783 , 0.26367681, 0.16835774, 0.65082042,
        0.33946951, 0.36490385, 0.1439133 , 0.42730295, 0.90308998,
        0.72078537, 0.71010536, 0.03236132, 0.49296547, 0.90570667,
        0.84678979, 0.30424041, 0.47319842, 0.65222239, 0.16758264,
        0.86305991, 0.29808536, 0.98047031, 0.16287792, 0.89384153,
        0.58076195, 0.50207889, 0.2878743 , 0.22041037, 0.04599868,
        0.2656977 , 0.16105866, 0.35016142, 0.90735084, 0.59606302,
        0.23005038, 0.01238927, 0.75210044, 0.48210542, 0.79442372,
        0.53036646, 0.29861608, 0.14784297, 0.02

In [45]:
labels[:10]

array([12.2376462 ,  7.34984909,  4.72351777, 27.79622211, 35.45494339,
       17.18490062, 35.0863021 , 47.93613287, 29.53286487,  7.85463053])

In [46]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
# 1 neuron bo chcemy przwidywaną wartość ciągłą.
# Funkcja aktywacji domyślnie jest liniowa a(x)
model.add(Dense(units=1))

model.compile(optimizer='rmsprop',
              loss='mse')

model.fit(data, labels, epochs=30, batch_size=32, validation_split=0.2)

Epoch 1/30
25/25 [==============================] - 0s 8ms/step - loss: 671.5061 - val_loss: 424.4634
Epoch 2/30
25/25 [==============================] - 0s 3ms/step - loss: 320.1439 - val_loss: 231.4867
Epoch 3/30
25/25 [==============================] - 0s 3ms/step - loss: 240.5469 - val_loss: 193.1378
Epoch 4/30
25/25 [==============================] - 0s 3ms/step - loss: 225.3478 - val_loss: 191.0755
Epoch 5/30
25/25 [==============================] - 0s 3ms/step - loss: 215.7509 - val_loss: 190.6980
Epoch 6/30
25/25 [==============================] - 0s 3ms/step - loss: 214.1038 - val_loss: 188.4960
Epoch 7/30
25/25 [==============================] - 0s 3ms/step - loss: 208.7536 - val_loss: 196.7604
Epoch 8/30
25/25 [==============================] - 0s 3ms/step - loss: 218.8714 - val_loss: 191.3250
Epoch 9/30
25/25 [==============================] - 0s 3ms/step - loss: 211.6386 - val_loss: 187.4020
Epoch 10/30
25/25 [==============================] - 0s 3ms/step - loss: 210.3236 

In [47]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(units=1))

model.compile(optimizer='rmsprop',
              loss='mae',
              metrics=['mse'])

model.fit(data, labels, epochs=30, batch_size=32, validation_split=0.2)

Epoch 1/30
25/25 [==============================] - 1s 8ms/step - loss: 22.3192 - mse: 703.4715 - val_loss: 18.0328 - val_mse: 456.5960
Epoch 2/30
25/25 [==============================] - 0s 3ms/step - loss: 15.8601 - mse: 370.4224 - val_loss: 14.1166 - val_mse: 267.0532
Epoch 3/30
25/25 [==============================] - 0s 4ms/step - loss: 13.3946 - mse: 245.3792 - val_loss: 12.4568 - val_mse: 200.2270
Epoch 4/30
25/25 [==============================] - 0s 3ms/step - loss: 12.6718 - mse: 212.2326 - val_loss: 12.2241 - val_mse: 193.1539
Epoch 5/30
25/25 [==============================] - 0s 3ms/step - loss: 13.0933 - mse: 225.3273 - val_loss: 12.0728 - val_mse: 189.5040
Epoch 6/30
25/25 [==============================] - 0s 3ms/step - loss: 13.0384 - mse: 224.1393 - val_loss: 12.0117 - val_mse: 188.2601
Epoch 7/30
25/25 [==============================] - 0s 3ms/step - loss: 12.8627 - mse: 216.7516 - val_loss: 12.1375 - val_mse: 190.7274
Epoch 8/30
25/25 [==============================

In [48]:
# 10 próbek testowych, każda o wymiarze 150
test_data = np.random.random((10, 150))

model.predict(test_data)

array([[24.573427],
       [28.470768],
       [30.131481],
       [23.567585],
       [25.895725],
       [24.743126],
       [28.2812  ],
       [26.498436],
       [24.990889],
       [21.373272]], dtype=float32)